In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:14 http://ppa.launchpad.net/c2d4u.team/

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-01-22 18:13:31--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.26MB/s    in 0.2s    

2023-01-22 18:13:31 (6.26 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [30]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)


### Create DataFrames to match tables

In [31]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     650634| R3EQSTM9PWRAL|B0091LBZSU|     578484426|Demograss Capsule...|Health & Personal...|          3|            0|          0|   N|                Y|         Three Stars|Only came with 30...| 2015-08-31|
|         US|   19827510| RBWPRK17XKIXD|B00PWW3LQ6|     456433146|Viva Labs #1 Prem...|Health & Personal

In [32]:
# Create the customers_table DataFrame
# customers_df = df.groupby("").agg({""}).withColumnRenamed("", "customer_count")
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    4919528|             5|
|   51451778|             3|
|   12713799|             1|
|    8673341|             1|
|   42146698|             2|
|    1117644|             2|
|   28058398|             1|
|   14375645|             1|
|   24540309|             1|
|   39715602|             1|
|     654272|             1|
|    9015608|             5|
|   43920023|            42|
|   38209321|             1|
|   12328685|             1|
|    2866605|             2|
|   38273165|             5|
|   41066514|             1|
|   28377689|             2|
|   17018784|             2|
+-----------+--------------+
only showing top 20 rows



In [49]:
df.select("customer_count")

DataFrame[customer_count: bigint]

In [35]:
# Create the products_table DataFrame and drop duplicates. 
# products_df = df.select([]).drop_duplicates()
products_df = df.select(["product_id", "product_title"]).drop_duplicates(["product_id"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0000045284|Loreal Paris Kaja...|
|0000800309|Himalaya Clarina ...|
|0004124677|  The Workbench Book|
|0004720962|    SAS Self-Defense|
|0006386830|Leslie Nielsen's ...|
|0006726909|Spanish Large Pri...|
|0007223099|Rani and the Thre...|
|0007241887|The Return to Nar...|
|0007622880|                Mine|
|0021500339|                Ohio|
|0064401251|It Can't Hurt For...|
|0072884975|THERMODYNAMICS-PR...|
|0077107950|Services Marketin...|
|0077663004|Contemporary Nutr...|
|0077676920|Contemporary Nutr...|
|0077862333|Auditing and Assu...|
|0091920655|     The Ghost Agent|
|0099252716|The Voyage of the...|
|0099274175|   Shadows of Empire|
|0140018654|Exile and the Kin...|
+----------+--------------------+
only showing top 20 rows



In [51]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
# review_id_df = df.select([, to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df = df.select(["review_id","customer_id","product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")]).drop_duplicates(["review_id"])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R10000VJGO8MCK|    2083334|B00DU54RL0|     340307372| 2014-08-05|
|R10007LUZ5DLQQ|   52244216|B004HHPWMS|     751364015| 2014-11-28|
|R10008CAT4CGNA|   52760682|B000E95HP0|     890646754| 2014-07-25|
|R1000AJEXL9Q32|     960113|B00N3LR4JS|      30647045| 2015-06-12|
|R1000J3TT2MYV3|   50861671|B0000Y8H2E|     141660050| 2006-01-26|
|R1000JDI0VDXQD|   38376753|B00F6WRW2Q|     289111448| 2014-01-11|
|R1000MHSGYLONA|   20712206|B00J2RRECO|     365489360| 2015-01-24|
|R1000VN6RICGHJ|   44772349|B0068NH4CU|     695238852| 2015-08-21|
|R1000WJKFNFQ1G|   23294231|B00416V4AO|      33880966| 2012-03-12|
|R1001360AC1U6K|   47040625|B00CLEENWY|     123187169| 2013-10-10|
|R10017XOIVR8MQ|   52871366|B002JMEK94|     712646836| 2010-04-16|
|R100196DQTMCJS|   41683200|B00M4M314Y|     911019126| 2015-07

In [37]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R3EQSTM9PWRAL|          3|            0|          0|   N|                Y|
| RBWPRK17XKIXD|          5|            0|          0|   N|                Y|
| RRSLOAF273XFC|          5|            1|          2|   N|                Y|
|R3S8W9Q6SWIT8O|          4|            0|          0|   N|                Y|
|R3QQ6NSLRVBFJC|          4|            0|          0|   N|                Y|
|R2XYDBMHUVJCSX|          2|            0|          1|   N|                Y|
|R1L94ESVC657A9|          5|            0|          0|   N|                Y|
|R3DI4B8LDWFQ3K|          1|            3|          4|   N|                Y|
|R1G7VV4HCXUQQL|          5|            0|          0|   N|                Y|
|R3TTNYN1TAMUSU|          5|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [53]:
# Configure settings for RDS
from getpass import getpass
password = getpass('Enter database password')
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.csi5sptjejzy.us-east-1.rds.amazonaws.com:5432/amazon_reviews"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [55]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [45]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [46]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [47]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)